## Compute FIMs a Single Reach in a Region

The purpose of this notebook is to compute the FIMs for all reaches in each of our study areas.

In [1]:
import json
import pandas
import urllib3
import requests

import compute_rating_increments as cr

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

Get the COMIDs that we'll use as input to the FIM tools.

In [2]:
# the following indices correspond to the Flowlines layer

locations = {'DeSoto': 0,
             'MountAscutney': 0,
             'RoaringRiverStatePark':13,
             'SpringfieldGreeneCounty':1,
             'TwoRiversOttauquechee': 0,
             'Windham': 0}

In [3]:
base_url = "https://arcgis.cuahsi.org/arcgis/rest/services/CIROH-ComRes"

In [4]:
params = {
    'where': '1=1',
    'geometryType': 'esriGeometryEnvelope',
    'spatialRel': 'esriSpatialRelIntersects',
    'units': 'esriSRUnit_Foot',
    'outFields': 'COMID,REACHCODE',
    'returnGeometry': 'false',
    'returnDistinctValues': 'true', 
    'returnIdsOnly': 'false',
    'returnCountOnly': 'false',
    'returnExtentOnly': 'false',
    'returnZ': 'false',
    'returnM': 'false',
    'multipatchOption': 'xyFootprint',
    'returnTrueCurves': 'false',
    'returnExceededLimitFeatures': 'false',
    'returnCentroid': 'false',
    'timeReferenceUnknownClient': 'false',
    'sqlFormat': 'none',
    'featureEncoding': 'esriDefault', 
    'f': 'json' 
}

In [5]:
riverids = {}
for locname, layerid in locations.items():
    response = requests.get(f'{base_url}/{locname}/FeatureServer/{layerid}/query', params=params, verify=False)
    
    
    data = response.json()
    if 'error' in data.keys():
        print(f"\n * Error: {locname} - {data['error']['code']} - {data['error']['message']}\n")
        continue
    
    comids = []
    if 'features' in data:
        for feature in data['features']:
            if 'attributes' in feature and 'COMID' in feature['attributes']:
                comids.append({'reachid': feature['attributes']['COMID'],
                               'huc8': feature['attributes']['REACHCODE'][0:8]})

    riverids[locname] = comids
    print(f'Found {len(comids)} rivers in {locname}')

Found 551 rivers in DeSoto
Found 1163 rivers in MountAscutney
Found 401 rivers in RoaringRiverStatePark
Found 1704 rivers in SpringfieldGreeneCounty
Found 1945 rivers in TwoRiversOttauquechee
Found 1352 rivers in Windham


Compute the rating curve increments for one of these reaches

In [6]:
# get a river of interest
reachid = riverids['DeSoto'][0]['reachid']
hucid = riverids['DeSoto'][0]['huc8']

print('ReachID: HUC8')
print(f'{reachid}: {hucid}')

ReachID: HUC8
3629135: 07140101


## Generate Cloudrun Commands

In [7]:
riverids['DeSoto'][0]['huc8']

'07140101'

In [8]:
for r in riverids['DeSoto']:
    print(r)
    break

{'reachid': 3629135, 'huc8': '07140101'}


In [9]:
interval = 0.5
procs = 10
for locname, layerid in locations.items():
    with open(f'cloudrun-inputs-{locname}.txt', 'w') as f:
        for reach in riverids[locname]:
            cmd = f"reachfim_interval,{reach['huc8']},{reach['reachid']},{interval},{procs}"
            f.write(f'{cmd}\n')

In [31]:
stage, flow = cr.compute_rating_increments(huc_id = hucid,
                                           reach_id = reachid,
                                           increment = 0.5,
                                           verbose=False)

In [32]:
pandas.DataFrame({'stage_m': stage[1:],
                  'cms': flow[1:]})

,stage_m,cms
0,0.5,5.39
1,1.0,104.71
2,1.5,336.58
3,2.0,679.52
4,2.5,1112.53
5,3.0,1631.95
6,3.5,2231.61
7,4.0,2905.77
8,4.5,3642.82
9,5.0,4429.69


In [9]:
pandas.DataFrame({'stage_m': stage[1:],
                  'cms': flow[1:]}).to_csv(f'{reachid}.csv',
                                      index=False)

Generate FIM maps using these cached data.

In [10]:
df = pandas.read_csv(f'{reachid}.csv')
df

,stage_m,cms
0,0.5,5.39
1,1.0,104.71
2,1.5,336.58
3,2.0,679.52
4,2.5,1112.53
5,3.0,1631.95
6,3.5,2231.61
7,4.0,2905.77
8,4.5,3642.82
9,5.0,4429.69


In [11]:
# Generate Labels

flow_labels = [f"{str(v).replace('.','_')}_cms"
                  for v in df.cms.values]

stage_labels = [f"{str(v).replace('.','_')}_m"
                  for v in df['stage_m'].values]
labels = [f"{reachid}__{stage_labels[i]}__{flow_labels[i]}" for i in range(0, len(flow_labels))]

In [12]:
# generate inputs for test

num_fims_to_generate = 5

test_ids = ','.join([str(reachid)] * num_fims_to_generate) 
#test_ids

test_flows = ','.join([str(v) for v in df.cms.values][0:num_fims_to_generate])
#test_flows


!echo COMMAND:
!echo run.sh {hucid} {test_ids} {test_flows} {','.join(labels[0:num_fims_to_generate])}

COMMAND:
run.sh 07140101 3629135,3629135,3629135,3629135,3629135 5.39,104.71,336.58,679.52,1112.53 3629135__0_5_m__5_39_cms,3629135__1_0_m__104_71_cms,3629135__1_5_m__336_58_cms,3629135__2_0_m__679_52_cms,3629135__2_5_m__1112_53_cms


In [13]:
!run.sh {hucid} {test_ids[0]} {test_flows[0]} {','.join(labels[0:num_fims_to_generate])}

Repository already exists at /home/code/inundation-mapping and contains files. Skipping clone.
Data for HUC 07140101
Copied the /home/output/flood_07140101/07140101/branch_ids.csv to /home/output/flood_07140101/fim_inputs.csv as fim_inputs.csv.
Unique feature IDs saved to /home/output/flood_07140101/feature_IDs.csv.
Completed in 2.04 minutes.

/home/code/inundation-mapping/tools/inundation.py:340: NumbaTypeSafetyWarning: unsafe cast from int32 to int16. Precision may be lost.
  if catchments[i, j] in catchment_stages_dict:

Inundation mapping for 07140101 completed successfully.
╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /home/generate_fim.py:223 in generate_reach_fim                              │
│                                                                              │
│   220 │   │                                                                  │
│   221 │   │   # compute FIM using the moasic approach established            │
│   222 │